In [116]:
from bs4 import BeautifulSoup
import requests
import urllib
import re
import pandas as pd
from tabulate import tabulate
from urllib import request
import nltk
from nltk.tokenize import regexp_tokenize
from __future__ import division, unicode_literals

In [136]:
def getSoup(url):
    grab_page = requests.get(url,timeout=5)
    if grab_page.status_code != 200:
        return None
    else:
        return BeautifulSoup(grab_page.content, "html5lib")

def getBookdetails(childsoup):
    bookname=[]
    child_body = child_soup.find('div', id="download")
    link_table = child_body.table
    ## get title of book
    metalist=child_soup.head.find_all("meta")
    for m in metalist:
        if m.get("name", None) == "title":
            bookname.append(m.get("content", None))
    
    #htmllinklist=[]
    #utf8linklist=[]
    ## get book links
    for l in link_table.find_all('tr'):
        column=l.find_all('td')
        for rc in column:
            htmllink = rc.findAll(title="Download", type=re.compile("text/html"))
            if (htmllink):
                for h in htmllink:
                    if(h.attrs["href"]):
                        #htmllinklist.append(h.attrs["href"])
                        htmllinklist= h.attrs["href"]
                    else:
                        htmllinklist=None
            utf8link = rc.findAll(title="Download", type=re.compile("text/plain"))
            if (utf8link):
                for u in utf8link:
                    if(u.attrs["href"]):
                        utf8linklist= u.attrs["href"]
                    else:
                        utf8linklist=None
    #print(bookname)
    return bookname,htmllinklist,utf8linklist

## define reusable functions

def read_html(url):
    html = request.urlopen(url).read().decode('utf8')
    raw = BeautifulSoup(html, 'html.parser').get_text()
    tokens = word_tokenize(raw)
    text = nltk.Text(tokens)
    return text

def lexical_diversity(text):
    return len(set(text))/len(text)

def percentage(word,text):
    return 100* text.count(word)/len(text)

def read_utf8(url):
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    tokens = nltk.word_tokenize(raw)
    text = nltk.Text(tokens)
    return text

def vocab(tokens):
    return set(tokens)

def read_utf8_contentonly(url):
    response = request.urlopen(url)
    raw = response.read().decode('utf8')
    start=raw.find("CONTENTS")
    if start== -1:
        start=0
    #print(start)    
    if raw.rfind("End of the Project Gutenberg") != -1:
        end=raw.rfind("End of the Project Gutenberg")
    elif raw.rfind("End of Project Gutenberg's") != -1:
        end=raw.rfind("End of Project Gutenberg's")
    elif raw.rfind("END OF THE PROJECT GUTENBERG") != -1:
        end=raw.rfind("END OF THE PROJECT GUTENBERG")
    
    if end == -1:
        end=len(raw)
        
    #print(end)
    tokens = nltk.word_tokenize(raw[start:end])
    text = nltk.Text(tokens)
    return text


def text_norm(text):
    #remove numbers and punctuations and convert all words to lower case to remove duplicates later
    words=[word.lower() for word in text if word.isalpha()]
    return words
    
# define reusable functions for list of books

def createlist(booklist,norm):
    textread=[]
    docnames=[]
    if norm:
        for name,url in booklist.items():
            docnames.append(name)
            textread.append(text_norm(read_utf8_contentonly(url)))
    else:
        for name,url in booklist.items():
            docnames.append(name)
            textread.append(read_utf8(url))
    
    return docnames,textread

# find lexical diversity of the books
def lexdiv_list(textread):
    lexdiv=[]
    for tmp in textread:
        lexdiv.append(lexical_diversity(tmp))
    #for l in lexdiv:
    #    print(l)
    
    return lexdiv

# find vocab of books
def vocabsize_list(textread):
    vocabsize=[]
    for tmp in textread:
        vocabsize.append(len(vocab(tmp)))
    #for vc in vocabsize:
    #    print(vc)
    
    return vocabsize
    
# print tabular output
def tab_bookstats(book_pd):
    print(tabulate(book_pd, headers='keys', tablefmt='simple',showindex=False,))

In [98]:
# call the main URL
soup=getSoup("http://www.gutenberg.org/wiki/Children%27s_Instructional_Books_(Bookshelf)")

In [50]:
a=soup.find_all(class_=re.compile("toctext"))
c=[]
for main in a:
    c.append(main.text)
    
for child in c:
    print(child)

Readers
Misc.
Graded Readers
Poetry Readers
Readers for English Speakers Learning Other Languages
Non-English Readers
About Readers
Science and Nature
History
Geography
Uncategorized


In [102]:
# find section
bookname=[]
booklink=[]

section_list = soup.find(class_="mw-parser-output")
links = section_list.find_all('a', class_="extiw")

for l in links:
    bookname.append(l.contents)
    booklink.append(l.attrs["href"])


In [96]:

bnlist=[]
hllist=[]
ullist=[]

child_soup=getSoup("http://www.gutenberg.org/ebooks/19721")
bn,hl,ul=getBookdetails(child_soup)
bnlist.append(bn)
hllist.append(hl)
ullist.append(ul)


['The Literary World Seventh Reader by Browne, Metcalf, and Withers']


In [103]:
bnlist=[]
hllist=[]
ullist=[]

for l in booklink:
    print("http"+l)
    print("======================================================")
    child_soup=getSoup("http:"+l)
    bn,hl,ul=getBookdetails(child_soup)
    bnlist.append(bn)
    hllist.append(hl)
    ullist.append(ul)


http//www.gutenberg.org/ebooks/7841
http//www.gutenberg.org/ebooks/5742
http//www.gutenberg.org/ebooks/13539
http//www.gutenberg.org/ebooks/7425
http//www.gutenberg.org/ebooks/16046
http//www.gutenberg.org/ebooks/22420
http//www.gutenberg.org/ebooks/23424
http//www.gutenberg.org/ebooks/18217
http//www.gutenberg.org/ebooks/24053
http//www.gutenberg.org/ebooks/24644
http//www.gutenberg.org/ebooks/24703
http//www.gutenberg.org/ebooks/15659
http//www.gutenberg.org/ebooks/15170
http//www.gutenberg.org/ebooks/10811
http//www.gutenberg.org/ebooks/9106
http//www.gutenberg.org/ebooks/19721
http//www.gutenberg.org/ebooks/14640
http//www.gutenberg.org/ebooks/14668
http//www.gutenberg.org/ebooks/14766
http//www.gutenberg.org/ebooks/14880
http//www.gutenberg.org/ebooks/15040
http//www.gutenberg.org/ebooks/16751
http//www.gutenberg.org/ebooks/13853
http//www.gutenberg.org/ebooks/15825
http//www.gutenberg.org/ebooks/18561
http//www.gutenberg.org/ebooks/18702
http//www.gutenberg.org/ebooks/19923
http/

http//www.gutenberg.org/ebooks/22245
http//www.gutenberg.org/ebooks/23560
http//www.gutenberg.org/ebooks/23728
http//www.gutenberg.org/ebooks/22425
http//www.gutenberg.org/ebooks/22600
http//www.gutenberg.org/ebooks/22766
http//www.gutenberg.org/ebooks/22917
http//www.gutenberg.org/ebooks/12655
http//www.gutenberg.org/ebooks/22925
http//www.gutenberg.org/ebooks/23395
http//www.gutenberg.org/ebooks/24030
http//www.gutenberg.org/ebooks/24125
http//www.gutenberg.org/ebooks/24656
http//www.gutenberg.org/ebooks/20557


In [107]:
book_df=pd.DataFrame({'bookname' : bnlist,
                   'html-link' : hllist,
                   'utf8-link' : ullist})
print(tabulate(book_df, headers='keys', tablefmt='simple',showindex=False,))

bookname                                                                                 html-link                                            utf8-link
---------------------------------------------------------------------------------------  ---------------------------------------------------  -------------------------------------------
['A Primary Reader: Old-time Stories, Fairy Tales and Myths Retold by Children']         //www.gutenberg.org/files/7841/7841-h/7841-h.htm     //www.gutenberg.org/ebooks/7841.txt.utf-8
['The Bird-Woman of the Lewis and Clark Expedition by Katherine Chandler']               //www.gutenberg.org/ebooks/5742.html.noimages        //www.gutenberg.org/ebooks/5742.txt.utf-8
["Dr. Scudder's Tales for Little Readers, About the Heathen. by John Scudder"]           //www.gutenberg.org/files/13539/13539-h/13539-h.htm  //www.gutenberg.org/ebooks/13539.txt.utf-8
['The Louisa Alcott Reader: a Supplementary Reader for the Fourth Year of School']       //www.gutenberg.org/

In [137]:
lexdiv_list=[]
vocab_size_list=[]
for u in ullist:
    print("http:"+u)
    print("=====================================")
    t=text_norm(read_utf8_contentonly("http:"+u))
    lexdiv_list.append(lexical_diversity(t))
    vocab_size_list.append(len(vocab(t)))

book_df["lexical-diversity"]=lexdiv_list
book_df["vocab-size"]=vocab_size_list

print(tabulate(book_df, headers='keys', tablefmt='simple',showindex=False,))

http://www.gutenberg.org/ebooks/7841.txt.utf-8
http://www.gutenberg.org/ebooks/5742.txt.utf-8
http://www.gutenberg.org/ebooks/13539.txt.utf-8
http://www.gutenberg.org/ebooks/7425.txt.utf-8
http://www.gutenberg.org/ebooks/16046.txt.utf-8
http://www.gutenberg.org/ebooks/22420.txt.utf-8
http://www.gutenberg.org/files/23424/23424-0.txt
http://www.gutenberg.org/ebooks/18217.txt.utf-8
http://www.gutenberg.org/ebooks/24053.txt.utf-8
http://www.gutenberg.org/ebooks/24644.txt.utf-8
http://www.gutenberg.org/ebooks/24703.txt.utf-8
http://www.gutenberg.org/ebooks/15659.txt.utf-8
http://www.gutenberg.org/ebooks/15170.txt.utf-8
http://www.gutenberg.org/ebooks/10811.txt.utf-8
http://www.gutenberg.org/ebooks/9106.txt.utf-8
http://www.gutenberg.org/ebooks/19721.txt.utf-8
http://www.gutenberg.org/ebooks/14640.txt.utf-8
http://www.gutenberg.org/ebooks/14668.txt.utf-8
http://www.gutenberg.org/ebooks/14766.txt.utf-8
http://www.gutenberg.org/ebooks/14880.txt.utf-8
http://www.gutenberg.org/ebooks/15040.txt.u

http://www.gutenberg.org/files/22917/22917-0.txt
http://www.gutenberg.org/ebooks/12655.txt.utf-8
http://www.gutenberg.org/ebooks/22925.txt.utf-8
http://www.gutenberg.org/files/23395/23395-0.txt
http://www.gutenberg.org/ebooks/24030.txt.utf-8
http://www.gutenberg.org/ebooks/24125.txt.utf-8
http://www.gutenberg.org/ebooks/24656.txt.utf-8
http://www.gutenberg.org/ebooks/20557.txt.utf-8
bookname                                                                                 html-link                                            utf8-link                                      lexical-diversity    vocab-size
---------------------------------------------------------------------------------------  ---------------------------------------------------  -------------------------------------------  -------------------  ------------
['A Primary Reader: Old-time Stories, Fairy Tales and Myths Retold by Children']         //www.gutenberg.org/files/7841/7841-h/7841-h.htm     //www.gutenberg.org/ebooks/784